# Fase 4: Modeling - Clasification (Categorical)


In [ ]:
# Celda 1: Importar librerías y preparar datos

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing  import StandardScaler
from sklearn.pipeline        import Pipeline
from sklearn.linear_model    import LogisticRegression
from sklearn.svm             import SVC
from sklearn.tree            import DecisionTreeClassifier
from sklearn.ensemble        import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network  import MLPClassifier
from sklearn.metrics         import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Cargar datos
data_path = '/Users/luissalamanca/Desktop/Duoc/Machine/ML_Proyecto_Semestral/data/03_features/engineered_data.csv'
data = pd.read_csv(data_path, sep=';')

# Separar columnas concatenadas
if len(data.columns) == 1:
    column_name = data.columns[0]
    if ',' in column_name:
        new_columns = column_name.split(',')
        data_split = data[column_name].str.split(',', expand=True)
        data_split.columns = new_columns
        for col in data_split.columns:
            data_split[col] = pd.to_numeric(data_split[col], errors='coerce')
        data = data_split

# Crear variable objetivo multiclase
data['EffectivenessLevel'] = pd.cut(
    data['EffectivenessScore'].astype(float),
    bins=[-0.1, 0.5, 1.5, 5, np.inf],
    labels=['Bajo', 'Medio', 'Alto', 'Experto']
)

print(data['EffectivenessLevel'].value_counts())
print('\nEstadísticas de EffectivenessScore:\n', data['EffectivenessScore'].astype(float).describe())

# Visualizar distribución de las clases
plt.figure(figsize=(6, 3))
sns.countplot(x='EffectivenessLevel', data=data, order=['Bajo', 'Medio', 'Alto', 'Experto'])
plt.title('Distribución de clases: EffectivenessLevel')
plt.show()

# Lista de features y target
features = [
    'EconomicEfficiency',
    'EquipmentAdvantage',
    'KillAssistRatio',
    'StealthKillsRatio',
    'KDA'
]
X = data[features]
y = data['EffectivenessLevel']

# Dividir en entrenamiento/prueba (30% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Escalamiento de características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Convertir los arrays escalados en DataFrame para fácil indexación por nombre
X_train_scaled_df = pd.DataFrame(
    X_train_scaled,
    columns=features,
    index=X_train.index
)
X_test_scaled_df  = pd.DataFrame(
    X_test_scaled,
    columns=features,
    index=X_test.index
)

print("Formas de los datos:")
print(f"  X_train:           {X_train.shape}")
print(f"  X_test:            {X_test.shape}")
print(f"  X_train_scaled_df: {X_train_scaled_df.shape}")
print(f"  X_test_scaled_df:  {X_test_scaled_df.shape}")


In [ ]:
# Pipeline y parámetros para GridSearch
pipe_lr = Pipeline([
    ('clf', LogisticRegression(max_iter=1000, random_state=42))
])

param_grid_lr = {
    'clf__C': [0.1, 1, 10],
    'clf__solver': ['lbfgs', 'liblinear']
}

grid_lr = GridSearchCV(pipe_lr, param_grid_lr, cv=5, scoring='f1', n_jobs=-1)
grid_lr.fit(X_train_scaled_df, y_train)

# Predicciones y métricas
y_pred_lr = grid_lr.predict(X_test_scaled_df)

print("Mejores parámetros:", grid_lr.best_params_)
print(classification_report(y_test, y_pred_lr, digits=3))

cm = confusion_matrix(y_test, y_pred_lr)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Matriz de confusión - Logistic Regression")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.show()


In [ ]:
pipe_rf = Pipeline([
    ('clf', RandomForestClassifier(random_state=42))
])

param_grid_rf = {
    'clf__n_estimators': [50, 100],
    'clf__max_depth': [None, 10, 20]
}

grid_rf = GridSearchCV(pipe_rf, param_grid_rf, cv=3, scoring='f1', n_jobs=-1)
grid_rf.fit(X_train_scaled_df, y_train)

y_pred_rf = grid_rf.predict(X_test_scaled_df)

print("Mejores parámetros:", grid_rf.best_params_)
print(classification_report(y_test, y_pred_rf, digits=3))

cm = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Matriz de confusión - Random Forest")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.show()


In [ ]:
pipe_svc = Pipeline([
    ('clf', SVC())
])

param_grid_svc = {
    'clf__C': [0.1, 1, 10],
    'clf__kernel': ['linear', 'rbf']
}

grid_svc = GridSearchCV(pipe_svc, param_grid_svc, cv=3, scoring='f1', n_jobs=-1)
grid_svc.fit(X_train_scaled_df, y_train)

y_pred_svc = grid_svc.predict(X_test_scaled_df)

print("Mejores parámetros:", grid_svc.best_params_)
print(classification_report(y_test, y_pred_svc, digits=3))

cm = confusion_matrix(y_test, y_pred_svc)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Matriz de confusión - SVC")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.show()


In [ ]:
pipe_gb = Pipeline([
    ('clf', GradientBoostingClassifier(random_state=42))
])

param_grid_gb = {
    'clf__n_estimators': [50, 100],
    'clf__learning_rate': [0.05, 0.1, 0.2]
}

grid_gb = GridSearchCV(pipe_gb, param_grid_gb, cv=3, scoring='f1', n_jobs=-1)
grid_gb.fit(X_train_scaled_df, y_train)

y_pred_gb = grid_gb.predict(X_test_scaled_df)

print("Mejores parámetros:", grid_gb.best_params_)
print(classification_report(y_test, y_pred_gb, digits=3))

cm = confusion_matrix(y_test, y_pred_gb)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Matriz de confusión - Gradient Boosting")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.show()


In [ ]:
pipe_mlp = Pipeline([
    ('clf', MLPClassifier(max_iter=300, random_state=42))
])

param_grid_mlp = {
    'clf__hidden_layer_sizes': [(50,), (100,)],
    'clf__activation': ['relu', 'tanh']
}

grid_mlp = GridSearchCV(pipe_mlp, param_grid_mlp, cv=3, scoring='f1', n_jobs=-1)
grid_mlp.fit(X_train_scaled_df, y_train)

y_pred_mlp = grid_mlp.predict(X_test_scaled_df)

print("Mejores parámetros:", grid_mlp.best_params_)
print(classification_report(y_test, y_pred_mlp, digits=3))

cm = confusion_matrix(y_test, y_pred_mlp)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Matriz de confusión - MLPClassifier")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.show()


In [ ]:
# Análisis comparativo de modelos de clasificación disponibles
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Verificar qué modelos están disponibles
modelos_disponibles = {}

# Verificar Logistic Regression
try:
    if 'y_pred_lr' in globals() and 'grid_lr' in globals():
        modelos_disponibles['Logistic Regression'] = {
            'predicciones': y_pred_lr,
            'mejor_modelo': grid_lr.best_estimator_,
            'mejores_params': grid_lr.best_params_,
            'mejor_score': grid_lr.best_score_
        }
except NameError:
    pass

# Verificar Random Forest
try:
    if 'y_pred_rf' in globals() and 'grid_rf' in globals():
        modelos_disponibles['Random Forest'] = {
            'predicciones': y_pred_rf,
            'mejor_modelo': grid_rf.best_estimator_,
            'mejores_params': grid_rf.best_params_,
            'mejor_score': grid_rf.best_score_
        }
except NameError:
    pass

# Verificar SVC
try:
    if 'y_pred_svc' in globals() and 'grid_svc' in globals():
        modelos_disponibles['SVC'] = {
            'predicciones': y_pred_svc,
            'mejor_modelo': grid_svc.best_estimator_,
            'mejores_params': grid_svc.best_params_,
            'mejor_score': grid_svc.best_score_
        }
except NameError:
    pass

# Verificar Gradient Boosting
try:
    if 'y_pred_gb' in globals() and 'grid_gb' in globals():
        modelos_disponibles['Gradient Boosting'] = {
            'predicciones': y_pred_gb,
            'mejor_modelo': grid_gb.best_estimator_,
            'mejores_params': grid_gb.best_params_,
            'mejor_score': grid_gb.best_score_
        }
except NameError:
    pass

# Verificar MLP Classifier
try:
    if 'y_pred_mlp' in globals() and 'grid_mlp' in globals():
        modelos_disponibles['MLP Classifier'] = {
            'predicciones': y_pred_mlp,
            'mejor_modelo': grid_mlp.best_estimator_,
            'mejores_params': grid_mlp.best_params_,
            'mejor_score': grid_mlp.best_score_
        }
except NameError:
    pass

print(f"Modelos disponibles para análisis: {list(modelos_disponibles.keys())}")
print(f"Total de modelos: {len(modelos_disponibles)}")

if len(modelos_disponibles) == 0:
    print("⚠️ No se encontraron modelos entrenados. Asegúrate de ejecutar las celdas de entrenamiento primero.")
else:
    # Calcular métricas para cada modelo disponible
    metricas_comparacion = []

    for nombre_modelo, datos in modelos_disponibles.items():
        y_pred = datos['predicciones']
        
        metricas = {
            'Modelo': nombre_modelo,
            'Accuracy': accuracy_score(y_test, y_pred),
            'Precision': precision_score(y_test, y_pred, average='weighted'),
            'Recall': recall_score(y_test, y_pred, average='weighted'),
            'F1-Score': f1_score(y_test, y_pred, average='weighted'),
            'F1-Score CV': datos['mejor_score']  # Score de validación cruzada
        }
        metricas_comparacion.append(metricas)

    # Crear DataFrame con resultados
    df_resultados = pd.DataFrame(metricas_comparacion)
    df_resultados = df_resultados.round(4)

    print("\n" + "=" * 80)
    print("ANÁLISIS COMPARATIVO DE MODELOS DE CLASIFICACIÓN")
    print("=" * 80)
    print()

    # Mostrar tabla de resultados
    print("MÉTRICAS DE RENDIMIENTO:")
    print("-" * 80)
    print(df_resultados.to_string(index=False))
    print()

    # Encontrar el mejor modelo por cada métrica
    print("MEJORES MODELOS POR MÉTRICA:")
    print("-" * 40)
    for metrica in ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'F1-Score CV']:
        mejor_idx = df_resultados[metrica].idxmax()
        mejor_modelo = df_resultados.iloc[mejor_idx]['Modelo']
        mejor_valor = df_resultados.iloc[mejor_idx][metrica]
        print(f"{metrica:15}: {mejor_modelo:20} ({mejor_valor:.4f})")

    print()

    # Ranking general (promedio de métricas normalizadas)
    metricas_numericas = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'F1-Score CV']
    df_normalizado = df_resultados.copy()

    for metrica in metricas_numericas:
        df_normalizado[metrica] = (df_normalizado[metrica] - df_normalizado[metrica].min()) / \
                                  (df_normalizado[metrica].max() - df_normalizado[metrica].min())

    df_normalizado['Score_Promedio'] = df_normalizado[metricas_numericas].mean(axis=1)
    df_ranking = df_normalizado[['Modelo', 'Score_Promedio']].sort_values('Score_Promedio', ascending=False)

    print("RANKING GENERAL DE MODELOS:")
    print("-" * 35)
    for i, (_, row) in enumerate(df_ranking.iterrows(), 1):
        print(f"{i}. {row['Modelo']:20} (Score: {row['Score_Promedio']:.4f})")

    print()

    # Mostrar mejores parámetros del modelo ganador
    mejor_modelo_nombre = df_ranking.iloc[0]['Modelo']
    mejores_params = modelos_disponibles[mejor_modelo_nombre]['mejores_params']

    print(f"MEJORES PARÁMETROS DEL MODELO GANADOR ({mejor_modelo_nombre}):")
    print("-" * 60)
    for param, valor in mejores_params.items():
        print(f"{param}: {valor}")

    print()

    # Visualización comparativa (solo si hay más de un modelo)
    if len(modelos_disponibles) > 1:
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))

        # Gráfico de barras - Métricas principales
        metricas_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
        df_plot = df_resultados.set_index('Modelo')[metricas_plot]
        df_plot.plot(kind='bar', ax=axes[0,0], width=0.8)
        axes[0,0].set_title('Comparación de Métricas por Modelo')
        axes[0,0].set_ylabel('Score')
        axes[0,0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        axes[0,0].tick_params(axis='x', rotation=45)

        # Gráfico de F1-Score
        theta = range(len(df_resultados))
        f1_scores = df_resultados['F1-Score']
        axes[0,1].bar(theta, f1_scores, color='skyblue', alpha=0.7)
        axes[0,1].set_title('F1-Score por Modelo')
        axes[0,1].set_xticks(theta)
        axes[0,1].set_xticklabels(df_resultados['Modelo'], rotation=45, ha='right')
        axes[0,1].set_ylabel('F1-Score')

        # Heatmap de correlación entre métricas
        corr_matrix = df_resultados[metricas_numericas].corr()
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, ax=axes[1,0])
        axes[1,0].set_title('Correlación entre Métricas')

        # Distribución de scores
        df_resultados[metricas_plot].boxplot(ax=axes[1,1])
        axes[1,1].set_title('Distribución de Métricas')
        axes[1,1].set_ylabel('Score')
        axes[1,1].tick_params(axis='x', rotation=45)

        plt.tight_layout()
        plt.show()
    else:
        # Gráfico simple para un solo modelo
        metricas_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
        valores = [df_resultados.iloc[0][metrica] for metrica in metricas_plot]
        
        plt.figure(figsize=(10, 6))
        plt.bar(metricas_plot, valores, color='skyblue', alpha=0.7)
        plt.title(f'Métricas del Modelo: {df_resultados.iloc[0]["Modelo"]}')
        plt.ylabel('Score')
        plt.ylim(0, 1)
        for i, v in enumerate(valores):
            plt.text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
        plt.show()

    # Conclusiones finales
    print("=" * 80)
    print("CONCLUSIONES FINALES")
    print("=" * 80)

    if len(modelos_disponibles) > 1:
        # Análisis comparativo
        mejor_f1 = df_resultados.loc[df_resultados['F1-Score'].idxmax()]
        peor_f1 = df_resultados.loc[df_resultados['F1-Score'].idxmin()]

        print(f"""
📊 RESUMEN EJECUTIVO:
• Mejor modelo general: {mejor_modelo_nombre}
• Mejor F1-Score: {mejor_f1['Modelo']} ({mejor_f1['F1-Score']:.4f})
• Mayor Accuracy: {df_resultados.loc[df_resultados['Accuracy'].idxmax(), 'Modelo']} ({df_resultados['Accuracy'].max():.4f})

🔍 ANÁLISIS DETALLADO:
• Diferencia entre mejor y peor F1-Score: {mejor_f1['F1-Score'] - peor_f1['F1-Score']:.4f}
• Modelo más estable: {df_resultados.loc[(df_resultados['F1-Score'] - df_resultados['F1-Score CV']).abs().idxmin(), 'Modelo']}
• Promedio general F1-Score: {df_resultados['F1-Score'].mean():.4f}

💡 RECOMENDACIONES:
• Para producción: Usar {mejor_modelo_nombre} por su rendimiento general superior
• Considerar tiempo de entrenamiento e inferencia
• Validar con datos no vistos antes del despliegue
""")
    else:
        # Análisis de modelo único
        modelo_unico = df_resultados.iloc[0]
        print(f"""
📊 ANÁLISIS DEL MODELO: {modelo_unico['Modelo']}

🔍 MÉTRICAS DE RENDIMIENTO:
• Accuracy: {modelo_unico['Accuracy']:.4f}
• Precision: {modelo_unico['Precision']:.4f}
• Recall: {modelo_unico['Recall']:.4f}
• F1-Score: {modelo_unico['F1-Score']:.4f}
• F1-Score CV: {modelo_unico['F1-Score CV']:.4f}

💡 EVALUACIÓN:
• Diferencia entre test y CV: {abs(modelo_unico['F1-Score'] - modelo_unico['F1-Score CV']):.4f}
• Rendimiento general: {'Excelente' if modelo_unico['F1-Score'] > 0.9 else 'Bueno' if modelo_unico['F1-Score'] > 0.8 else 'Aceptable' if modelo_unico['F1-Score'] > 0.7 else 'Necesita mejora'}
""")

    print("\n⚠️  CONSIDERACIONES IMPORTANTES:")
    print("• Evaluar el modelo con datos completamente nuevos antes de producción")
    print("• Considerar la interpretabilidad según el caso de uso")
    print("• Monitorear el rendimiento en producción")
    print("• Considerar el tiempo de entrenamiento e inferencia")